In [1]:
%pip install "sentence-transformers[train]" wandb codecarbon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.9/494.9 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvi

In [2]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator



This is the ID of the synthetic dataset we generated previously.

In [3]:
dataset_id = "davanstrien/similarity-dataset-sc2-8b"

In [34]:
# 1. Load a model to finetune with 2. (Optional) model card data

dataset_name = dataset_id.split('/')[-1]

model = SentenceTransformer(
    "microsoft/mpnet-base",
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        train_datasets=[{"name": dataset_name, "id": dataset_id}],
        model_name=f"MPNet base trained on {dataset_name}",
    )
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [43]:
# 3. Load a dataset to finetune on
dataset = load_dataset(dataset_id,split='train')
dataset

Dataset({
    features: ['anchor', 'positive', 'negative', 'generation'],
    num_rows: 2324
})

In [44]:
dataset = dataset.train_test_split(test_size=0.2)

In [45]:
dataset

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative', 'generation'],
        num_rows: 1859
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative', 'generation'],
        num_rows: 465
    })
})

In [46]:
# 4. Define a loss function
loss = MultipleNegativesRankingLoss(model)

In [48]:
# 5. (Optional) Specify training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=f"models/{dataset_name}",
    # Optional training parameters:
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name=dataset_id,  # Will be used in W&B if `wandb` is installed
)


In [49]:
# 6. (Optional) Create an evaluator & evaluate the base model
dev_evaluator = TripletEvaluator(
    anchors=dataset['test']["anchor"],
    positives=dataset['test']["positive"],
    negatives=dataset['test']["negative"],
    name="code-prompt-similarity-dev",
)
dev_evaluator(model)

{'code-prompt-similarity-dev_cosine_accuracy': 0.8279569892473119,
 'code-prompt-similarity-dev_dot_accuracy': 0.4258064516129032,
 'code-prompt-similarity-dev_manhattan_accuracy': 0.864516129032258,
 'code-prompt-similarity-dev_euclidean_accuracy': 0.832258064516129,
 'code-prompt-similarity-dev_max_accuracy': 0.864516129032258}

In [50]:
# 7. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    loss=loss,
    evaluator=dev_evaluator,
)

In [51]:
trainer.train()

Step,Training Loss,Validation Loss,Code-prompt-similarity-dev Cosine Accuracy,Code-prompt-similarity-dev Dot Accuracy,Code-prompt-similarity-dev Manhattan Accuracy,Code-prompt-similarity-dev Euclidean Accuracy,Code-prompt-similarity-dev Max Accuracy
100,0.859900,0.159901,0.950538,0.043011,0.941935,0.946237,0.950538
200,0.191400,0.123644,0.961290,0.032258,0.963441,0.963441,0.963441
300,0.080600,0.131011,0.959140,0.043011,0.959140,0.959140,0.959140
400,0.038000,0.111195,0.965591,0.032258,0.961290,0.963441,0.965591
500,0.018300,0.126285,0.963441,0.034409,0.961290,0.963441,0.963441
600,0.007300,0.134239,0.956989,0.038710,0.954839,0.952688,0.956989
700,0.003600,0.133716,0.956989,0.038710,0.961290,0.956989,0.961290
800,0.002100,0.123739,0.959140,0.038710,0.954839,0.954839,0.959140
900,0.001300,0.129441,0.956989,0.040860,0.956989,0.956989,0.956989
1000,0.001100,0.132313,0.961290,0.038710,0.956989,0.959140,0.961290


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

...

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

TrainOutput(global_step=1170, training_loss=0.10300025749219278, metrics={'train_runtime': 409.1158, 'train_samples_per_second': 45.439, 'train_steps_per_second': 2.86, 'total_flos': 0.0, 'train_loss': 0.10300025749219278, 'epoch': 10.0})

In [52]:
# (Optional) Evaluate the trained model on the test set
test_evaluator = TripletEvaluator(
    anchors=dataset['test']["anchor"],
    positives=dataset['test']["positive"],
    negatives=dataset['test']["negative"],
)
test_evaluator(model)

{'cosine_accuracy': 0.9612903225806452,
 'dot_accuracy': 0.03870967741935484,
 'manhattan_accuracy': 0.9612903225806452,
 'euclidean_accuracy': 0.9612903225806452,
 'max_accuracy': 0.9612903225806452}

In [ ]:
# (Optional) Push it to the Hugging Face Hub
model.push_to_hub("code-prompt-similarity-model")